In [1]:
import torch 
from egg.zoo.pop.utils import get_common_opts, metadata_opener, load_from_checkpoint
from egg.zoo.pop.games import build_game
import hub
from torchvision import transforms

#from egg.zoo.pop.data import ImageTransformation
import numpy as np

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.0.1+cu118 with CUDA 1108 (you have 1.9.0+cu111)
    Python  3.8.16 (you have 3.8.5)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


In [2]:
model_path = "/home/mmahaut/projects/exps/tmlr/vocab_64/590132/final.tar"
metadata_path = "/home/mmahaut/projects/exps/tmlr/vocab_64/590132/wandb/latest-run/files/wandb-metadata.json"

In [3]:
opts = None
with open(metadata_path) as f:
    opts = get_common_opts(metadata_opener(f, data_type="wandb", verbose=True))
print(opts)

Namespace(additional_receivers='[]', additional_senders='[]', augmentation_type=None, aux_loss=None, aux_loss_weight=0.0, base_checkpoint_path='', batch_size=64, block_com_layer=False, checkpoint_dir='/home/mmahaut/projects/exps/tmlr/vocab_64', checkpoint_freq=0, com_channel='continuous', continuous_com=True, cuda=True, dataset_dir='/projects/colt/ILSVRC2012/ILSVRC2012_img_val/', dataset_name='imagenet_val', device=device(type='cuda'), distributed_context=DistributedContext(is_distributed=False, rank=0, local_rank=0, world_size=1, mode='none'), distributed_port=18363, force_gumbel=False, fp16=False, gs_temperature=5.0, gs_temperature_decay=1.0, image_size=384, is_single_class_batch=False, keep_classification_layer=False, load_from_checkpoint=None, lr=0.0001, max_len=1, minimum_gs_temperature=1.0, n_epochs=25, no_cuda=False, noisy_channel=None, non_linearity='sigmoid', num_workers=4, optimizer='adam', path_to_kmeans=None, pdb=False, preemptable=False, random_seed=111, recv_hidden_dim=20

In [4]:
pop_game = build_game(opts)
load_from_checkpoint(pop_game, model_path)
senders = pop_game.agents_loss_sampler.senders
for sender in senders:
    sender.eval()
    # TODO : remove gradients

initialize module vit
['vit']
['vit']
# loading trainer state from /home/mmahaut/projects/exps/tmlr/vocab_64/590132/final.tar


In [7]:
ds = hub.load("hub://activeloop/places205")

/

Opening dataset in read-only mode as you don't have write permissions.


\

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/places205



-

hub://activeloop/places205 loaded successfully.



In [5]:
class ImageTransformation:
    """
    resize and normalize
    """
    def __init__(self, size: int):
        transformations = [
                transforms.Resize(size=(size, size)),
            ]
        transformations.append(
                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
            )
        self.transform = transforms.Compose(transformations)

    def __call__(self, x):
        return self.transform(x)

In [8]:
size = 384
transformations = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(int(3/x.shape[0]), 1, 1)),
    transforms.Resize(size=(size, size)),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

def collate_fn(batch):
    return (
        torch.stack([x["images"] for x in batch], dim=0),
        torch.stack([torch.Tensor(x["labels"]).long() for x in batch], dim=0),
        torch.stack([torch.Tensor(x["index"]) for x in batch], dim=0),
    )
dataloader = ds.pytorch(num_workers = 0, shuffle = True, batch_size= 128, collate_fn=collate_fn, transform={'images':transformations,'labels':None, 'index':None})


/home/mmahaut/.local/lib/python3.8/site-packages/deeplake/integrations/pytorch/common.py:126: UserWarning: Decode method for tensors ['images'] is defaulting to numpy. Please consider specifying a decode_method in .pytorch() that maximizes the data preprocessing speed based on your transformation.
  warnings.warn(


In [9]:
class LinearClassifier(torch.nn.Module):
  def __init__(self, input_dim=2, output_dim=3):
    super(LinearClassifier, self).__init__()
    self.linear = torch.nn.Linear(input_dim, output_dim)

  def forward(self, x):
    x = self.linear(x)
    return x

criterion = torch.nn.CrossEntropyLoss()

classifiers = [LinearClassifier(16, 245).to("cuda")] * len(senders) # keeping all classifiers on cpu
classifiers.append(LinearClassifier(16, 245).to("cuda")) # <-shuffled-classifier : input a random message from one of the senders

optimizers = [torch.optim.Adam(classifier.parameters(), lr=0.01) for classifier in classifiers]

In [ ]:
device="cuda"

def forward_backward(model_idx, input_images, labels):
    # everyone goes to selected device
    senders[model_idx].to(device)
    
    message = senders[model_idx](input_images)
    output = classifiers[model_idx](message)

    loss = criterion(output, labels.view(-1))
    loss.backward()

    optimizers[i].step()
    optimizers[i].zero_grad()
    
    senders[model_idx].to("cpu")

    return message, output, loss

for batch_idx, batch in enumerate(dataloader):
    _rand_sender = torch.randint(0, len(senders), (1,)).item() # chosing the shuffled input for shuffled-classifier
    images, labels, _ = batch
    images = images.to(device)
    labels = labels.to(device)
    
    for i in range(len(senders) - 1):
        _, output, loss = forward_backward(i, images, labels)
        if batch_idx % 1000 == 0 :
            print(f"acc_{i} : ", (labels.to(device) == output.argmax(dim=1)).float().mean())
        if i == _rand_sender:
            _, output, loss = forward_backward(-1, images, labels)
            if batch_idx % 1000 == 0 :
                print("acc_shuffled : ", (labels == output.argmax(dim=1)).float().mean())
    
    